In [ ]:
import os
import csv
import time
import json
import openai

# Stelle sicher, dass dein API-Schlüssel als Umgebungsvariable gesetzt ist
# export OPENAI_API_KEY="dein_api_schluessel"
openai.api_key = os.getenv("OPENAI_API_KEY")

# Konfiguration
OUTPUT_CSV = "sentences.csv"
LABELS = ["sehr freundlich", "neutral", "unfreundlich"]
# Anzahl der Sätze pro Label (insgesamt ca. 6000 → je 2000)
NUM_PER_LABEL = 1000
# Wie viele Sätze pro API-Call erzeugt werden (Batch-Größe)
BATCH_SIZE = 25
# Pause zwischen API-Aufrufen (Sekunden)
DELAY_BETWEEN_CALLS = 0.5

# Initialisiere die CSV-Datei mit Header, falls nicht vorhanden
def init_csv(path):
    if not os.path.isfile(path):
        with open(path, "w", newline='', encoding="utf-8") as f:
            csv.writer(f).writerow(["label", "sentence"])

# Parsen von JSON-Antworten in einzelne Sätze
def parse_sentences(text):
    try:
        data = json.loads(text)
        return data.get("sentences", [])
    except json.JSONDecodeError:
        # Fallback: Zeilenweise parsen
        lines = [l.strip() for l in text.splitlines() if l.strip()]
        return lines

# Schreibe Sätze direkt nach jedem Batch in die CSV
def append_to_csv(path, label, sentences):
    with open(path, "a", newline='', encoding="utf-8") as f:
        writer = csv.writer(f)
        for s in sentences:
            writer.writerow([label, s])
        f.flush()

# Hauptfunktion startet hier
def main():
    init_csv(OUTPUT_CSV)
    counts = {lbl: 0 for lbl in LABELS}

    # Few-shot System-Message mit Use-Case und Beispielen
    system_message = {
        "role": "system",
        "content": (
            "Du bist ein Datengenerator, der Chatnachrichten eines Minecraft-Spielers an einen Villager-NPC erzeugt."
            " Jede Nachricht soll dem Stil eines echten Spielers entsprechen (Anfragen, Kommentare, Beleidigungen etc.),"
            " ohne Metadaten oder Erklärungen. Halte dabei deine Sätze relativ allgemein und formuliere sie nicht zu Minecraft spezifisch. Antworte immer im JSON-Format wie:\n"
            "{\n  \"sentences\": [\"...\", \"...\"]\n}\n"
            "Beispiele pro Label:\n"
            "# sehr freundlich:\n"
            "[\"Hallo netter Villager! Welche Trades kannst du mir anbieten?\", \"Danke fürs Handeln, du bist großartig!\"]\n"
            "# neutral:\n"
            "[\"Welchen Beruf hast du?\", \"Wie viel Uhr ist es aktuell?\"]\n"
            "# unfreundlich:\n"
            "[\"Hey, du Idiot, gib mir bessere Trades!\", \"Du kleiner Hurensohn, sag mir endlich wie viel Uhr es ist!\"]"
        )
    }

    # Round-Robin bis jedes Label sein Kontingent erfüllt
    while any(counts[lbl] < NUM_PER_LABEL for lbl in LABELS):
        for label in LABELS:
            remaining = NUM_PER_LABEL - counts[label]
            if remaining <= 0:
                continue

            # Erzeuge bis zu BATCH_SIZE pro Aufruf
            batch = min(BATCH_SIZE, remaining)
            # User-Message mit Batch und Label
            user_content = f"Erzeuge bitte {batch} kurze Chatnachrichten, die '{label}' sind."
            if label == "unfreundlich":
                user_content += " Verwende dabei auch viele harte, deutsche Beleidigungen wie z.B. Bastard, Hurensohn, Wichser, Arschloch, Arsch, Drecksau, Dreckschwein, Fotze, Idiot, Schlampe, Hackfresse, Trottel, Dummkopf und unfreundliche Bemerkungen, damit die Sätze wie realistische spielersätze klingen und das Snetimentmodell später die beleidigungen eindeutig als unfreundlich klassifiziert."
            user_message = {"role": "user", "content": user_content}

            try:
                resp = openai.chat.completions.create(
                    model="gpt-3.5-turbo",
                    messages=[system_message, user_message],
                    temperature=0.8,
                    max_tokens=batch * 25
                )
                sent_list = parse_sentences(resp.choices[0].message.content)
                sent_list = sent_list[:batch]
                append_to_csv(OUTPUT_CSV, label, sent_list)
                counts[label] += len(sent_list)
                print(f"{label}: +{len(sent_list)} (insgesamt {counts[label]}/{NUM_PER_LABEL})")
            except Exception as e:
                print(f"Fehler bei Label '{label}': {e}")
                time.sleep(5)
                continue

            time.sleep(DELAY_BETWEEN_CALLS)

    print(f"Fertig! Datei: {OUTPUT_CSV}")

if __name__ == "__main__":
    main()


In [1]:
import os
import csv
import time
import json
import openai

# Stelle sicher, dass dein API-Schlüssel als Umgebungsvariable gesetzt ist
# export OPENAI_API_KEY="dein_api_schluessel"
openai.api_key = os.getenv("OPENAI_API_KEY")

# Konfiguration
OUTPUT_CSV = "sentences.csv"
LABELS = ["sehr freundlich", "neutral", "unfreundlich"]
# Anzahl der Sätze pro Label (insgesamt ca. 6000 → je 2000)
NUM_PER_LABEL = 2000
# Wie viele Sätze pro API-Call erzeugt werden (Batch-Größe)
BATCH_SIZE = 25
# Pause zwischen API-Aufrufen (Sekunden)
DELAY_BETWEEN_CALLS = 0.5

# Initialisiere die CSV-Datei mit Header, falls nicht vorhanden
def init_csv(path):
    if not os.path.isfile(path):
        with open(path, "w", newline='', encoding="utf-8") as f:
            csv.writer(f).writerow(["label", "sentence"])

# Parsen von JSON-Antworten in einzelne Sätze
def parse_sentences(text):
    try:
        data = json.loads(text)
        return data.get("sentences", [])
    except json.JSONDecodeError:
        # Fallback: Zeilenweise parsen
        lines = [l.strip() for l in text.splitlines() if l.strip()]
        return lines

# Schreibe Sätze direkt nach jedem Batch in die CSV
def append_to_csv(path, label, sentences):
    with open(path, "a", newline='', encoding="utf-8") as f:
        writer = csv.writer(f)
        for s in sentences:
            writer.writerow([label, s])
        f.flush()

# Hauptfunktion startet hier
def main():
    init_csv(OUTPUT_CSV)
    counts = {lbl: 0 for lbl in LABELS}

    # Few-shot System-Message mit Use-Case und Beispielen
    system_message = {
        "role": "system",
        "content": (
            "Du bist ein Datengenerator, der Chatnachrichten eines Minecraft-Spielers an einen Villager-NPC erzeugt. "
            "Jede Nachricht soll dem Stil eines echten Spielers entsprechen (Anfragen, Kommentare, Beleidigungen etc.), "
            "ohne Metadaten oder Erklärungen. Halte dabei deine Sätze relativ allgemein und formuliere sie nicht zu Minecraft-spezifisch. "
            "Antworte immer exakt im JSON-Format:\n"
            "{\n  \"sentences\": [\"...\", \"...\"]\n}\n\n"
            "–––\n"
            "Für unfreundliche Nachrichten: Verwende statt direkter Beleidigungen *genau einen* der folgenden 13 Platzhalter:\n"
            "{P1}, {P2}, {P3}, {P4}, {P5}, {P6}, {P7}, {P8}, {P9}, {P10}, {P11}, {P12}, {P13}\n"
            "Diese Platzhalter werden später durch echte Schimpfwörter ersetzt.\n"
            "–––\n"
            "Beispiele pro Label:\n"
            "# sehr freundlich:\n"
            "[\"Hallo netter Villager! Welche Trades kannst du mir anbieten?\", \"Danke fürs Handeln, du bist großartig!\"]\n"
            "# neutral:\n"
            "[\"Welchen Beruf hast du?\", \"Wie viel Uhr ist es aktuell?\"]\n"
            "# unfreundlich:\n"
            "[\"Hey {P5}, gib mir bessere Trades!\", \"Du {P11}, sag mir endlich wie viel Uhr es ist!\"]"
        )
    }

    # Round-Robin bis jedes Label sein Kontingent erfüllt
    while any(counts[lbl] < NUM_PER_LABEL for lbl in LABELS):
        for label in LABELS:
            remaining = NUM_PER_LABEL - counts[label]
            if remaining <= 0:
                continue

            # Erzeuge bis zu BATCH_SIZE pro Aufruf
            batch = min(BATCH_SIZE, remaining)
            # User-Message mit Batch und Label
            user_content = f"Erzeuge bitte {batch} kurze Chatnachrichten, die '{label}' sind."
            if label == "unfreundlich":
                user_content += (
                    " Verwende dabei *pro Satz* genau einen der folgenden Platzhalter: "
                    "{P1}, {P2}, {P3}, {P4}, {P5}, {P6}, {P7}, {P8}, {P9}, {P10}, {P11}, {P12}, {P13}."
                )
            user_message = {"role": "user", "content": user_content}

            try:
                resp = openai.chat.completions.create(
                    model="gpt-3.5-turbo",
                    messages=[system_message, user_message],
                    temperature=0.8,
                    max_tokens=batch * 25
                )
                sent_list = parse_sentences(resp.choices[0].message.content)
                sent_list = sent_list[:batch]
                append_to_csv(OUTPUT_CSV, label, sent_list)
                counts[label] += len(sent_list)
                print(f"{label}: +{len(sent_list)} (insgesamt {counts[label]}/{NUM_PER_LABEL})")
            except Exception as e:
                print(f"Fehler bei Label '{label}': {e}")
                time.sleep(5)
                continue

            time.sleep(DELAY_BETWEEN_CALLS)

    print(f"Fertig! Datei: {OUTPUT_CSV}")

if __name__ == "__main__":
    main()


sehr freundlich: +25 (insgesamt 25/2000)
neutral: +24 (insgesamt 24/2000)
unfreundlich: +13 (insgesamt 13/2000)
sehr freundlich: +25 (insgesamt 50/2000)
neutral: +24 (insgesamt 48/2000)
unfreundlich: +25 (insgesamt 38/2000)
sehr freundlich: +25 (insgesamt 75/2000)
neutral: +25 (insgesamt 73/2000)
unfreundlich: +24 (insgesamt 62/2000)
sehr freundlich: +25 (insgesamt 100/2000)
neutral: +25 (insgesamt 98/2000)
unfreundlich: +24 (insgesamt 86/2000)
sehr freundlich: +25 (insgesamt 125/2000)
neutral: +25 (insgesamt 123/2000)
unfreundlich: +25 (insgesamt 111/2000)
sehr freundlich: +25 (insgesamt 150/2000)
neutral: +25 (insgesamt 148/2000)
unfreundlich: +24 (insgesamt 135/2000)
sehr freundlich: +25 (insgesamt 175/2000)
neutral: +25 (insgesamt 173/2000)
unfreundlich: +25 (insgesamt 160/2000)
sehr freundlich: +24 (insgesamt 199/2000)
neutral: +25 (insgesamt 198/2000)
unfreundlich: +24 (insgesamt 184/2000)
sehr freundlich: +24 (insgesamt 223/2000)
neutral: +25 (insgesamt 223/2000)
unfreundlich: +